In [ ]:

import numpy as np
import tensorflow as tf
from sklearn.model_selection import ParameterSampler
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import time
import csv
import os
import pickle
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the hyperparameter grid with more epochs
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'learning_rate': [0.001, 0.01, 0.1],
    'dropout_rate': [0.0, 0.3, 0.5],
    'batch_size': [32, 64],
    'epochs': [50, 100],
    'activation': ['relu', 'tanh']
}

# Number of random combinations to evaluate
n_iter = 20

# Generate random combinations of parameters
param_list = list(ParameterSampler(param_grid, n_iter=n_iter, random_state=42))

# Load initial model
initial_model_path = 'initial_model.keras'
initial_model = tf.keras.models.load_model(initial_model_path)

# Load the resampled dataset
with open('resampled_data.pkl', 'rb') as f:
  resampled_data = pickle.load(f)

X_train_resampled = resampled_data['X_train_resampled']
y_train_resampled = resampled_data['y_train_resampled']
X_val = resampled_data['X_val']
y_val = resampled_data['y_val']
print("Loaded resampled dataset")

# Function to set the optimizer
def set_optimizer(optimizer, learning_rate):
    if optimizer == 'adam':
        return tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        return tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

# Function to evaluate a single combination of parameters
def evaluate_model(params, idx):
    checkpoint_path = "/content/drive/MyDrive/model_checkpoint.h5"
    training_log_path = "/content/drive/MyDrive/training_log.csv"

    # Load initial model and set new optimizer and learning rate
    model = tf.keras.models.load_model(initial_model_path)
    model.compile(optimizer=set_optimizer(params['optimizer'], params['learning_rate']),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(checkpoint_path, save_weights_only=True, save_freq='epoch')
    csv_logger = CSVLogger(training_log_path, append=True)

    start_time = time.time()
    history = model.fit(X_train_resampled, y_train_resampled, epochs=params['epochs'], batch_size=params['batch_size'],
                        validation_data=(X_val, y_val), verbose=1,
                        callbacks=[early_stopping, model_checkpoint, csv_logger],
                        initial_epoch=0)
    duration = time.time() - start_time

    val_accuracy = model.evaluate(X_val, y_val, verbose=0)[1]

    return val_accuracy, model, history, duration

# Initialize CSV file to log hyperparameter tuning results
if not os.path.exists('/content/drive/MyDrive/hyperparameter_tuning_log.csv'):
    with open('/content/drive/MyDrive/hyperparameter_tuning_log.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['optimizer', 'learning_rate', 'dropout_rate', 'batch_size', 'epochs', 'activation', 'val_accuracy'])

# Load the saved state if it exists
if os.path.exists('/content/drive/MyDrive/saved_state.pkl'):
    with open('/content/drive/MyDrive/saved_state.pkl', 'rb') as f:
        saved_state = pickle.load(f)
        best_score = saved_state['best_score']
        best_params = saved_state['best_params']
        best_model = saved_state['best_model']
        best_history = saved_state['best_history']
        idx_start = saved_state['idx_start']
else:
    best_score = 0
    best_params = None
    best_model = None
    best_history = None
    idx_start = 1

total_start_time = time.time()
for idx in range(idx_start, n_iter + 1):
    params = param_list[idx - 1]
    print(f"Evaluating params: {params}")

    score, model, history, duration = evaluate_model(params, idx)

    # Log the results
    with open('/content/drive/MyDrive/hyperparameter_tuning_log.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([params['optimizer'], params['learning_rate'],
                         params['dropout_rate'], params['batch_size'],
                         params['epochs'], params['activation'], score])

    print(f"Score for params {params}: {score}")
    print(f"Time taken for this iteration: {duration} seconds")

    if score > best_score:
        best_score = score
        best_params = params
        best_model = model
        best_history = history

    # Save the current state
    with open('/content/drive/MyDrive/saved_state.pkl', 'wb') as f:
        pickle.dump({
            'best_score': best_score,
            'best_params': best_params,
            'best_model': best_model,
            'best_history': best_history,
            'idx_start': idx + 1
        }, f)

    print(f"Files for iteration {idx} successfully saved to Google Drive.")

total_duration = time.time() - total_start_time
print(f"Total time taken for the hyperparameter tuning: {total_duration} seconds")

print(f"Best Score: {best_score}")
print(f"Best Parameters: {best_params}")

# Save the best model manually
best_model_path = '/content/drive/MyDrive/best_model.keras'
best_model.save(best_model_path)
